In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pylab as plt
import seaborn as sns
import gc

import lightgbm as lgb
import ngboost as ngb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

from scipy import stats

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth',100)
pd.set_option('display.width',100)
pd.set_option('display.max_info_rows',100)

In [2]:
gc.collect()

31

In [3]:
# train = pd.read_csv('./jet_simple_data/simple_train_R04_jet.csv')
# test = pd.read_csv('./jet_simple_data/simple_test_R04_jet.csv')
data = pd.read_feather('E:/jet_complex_data/data_.feather')
data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548097 entries, 0 to 548096
Data columns (total 1542 columns):
event_id                                            548097 non-null object
label                                               371377 non-null float32
count                                               548097 non-null int8
event_id_number_of_particles_in_this_jet_mean       548097 non-null float32
event_id_number_of_particles_in_this_jet_max        548097 non-null int8
event_id_number_of_particles_in_this_jet_min        548097 non-null int8
event_id_number_of_particles_in_this_jet_sum        548097 non-null int16
event_id_number_of_particles_in_this_jet_std        548097 non-null float32
event_id_number_of_particles_in_this_jet_skew       548097 non-null float32
event_id_number_of_particles_in_this_jet_kurt       548097 non-null float32
event_id_number_of_particles_in_this_jet_q1         548097 non-null float32
event_id_number_of_particles_in_this_jet_q3         548097 non-

In [4]:
data['count'].value_counts()

2     227084
3     164229
4      88351
5      40187
6      16698
7       6184
1       2338
8       2078
9        695
10       188
11        46
12        17
14         1
13         1
Name: count, dtype: int64

In [5]:
data['isTest'] = data['label'].isna()
train = data.loc[data['isTest']==False]
test = data.loc[data['isTest']==True]

In [6]:
label_map = dict(zip(np.sort(train['label'].unique()), range(len(train['label'].unique()))))
label_map_reverse = dict(zip(range(len(train['label'].unique())), np.sort(train['label'].unique())))

In [7]:
# train['label'] = train['label'].map(label_map)
imp_array = np.load('E:/jet_complex_data/imp.npy', allow_pickle=True)
imp_array
len(imp_array)

array(['event_id_jet_px_sum', 'event_id_jet_px_mean',
       'event_id_jet_id_particle_py_std_min', ...,
       'event_id_particle_category_13_min',
       'event_id_jet_id_particle_mass/pt_min_min',
       'event_id_jet_id_particle_mass_min_min'], dtype=object)

1540

In [8]:
len(imp_array[:1000:-1])

539

In [9]:
drop_feat = ['event_id']
for x in imp_array[:1530:-1]:
    drop_feat.append(x)
# for col in data.columns.to_list():
# #     if 'kurt' in col or 'q1'in col or 'q3'in col:
#     if 'q3'in col:
#         print(col)
#         drop_feat.append(col)

In [10]:
used_feat = [f for f in test.columns if f not in ['label', 'jet_id', 'isTest'] + list(drop_feat)]
len(used_feat)
used_feat

1531

['event_id_number_of_particles_in_this_jet_mean',
 'event_id_number_of_particles_in_this_jet_max',
 'event_id_number_of_particles_in_this_jet_min',
 'event_id_number_of_particles_in_this_jet_sum',
 'event_id_number_of_particles_in_this_jet_std',
 'event_id_number_of_particles_in_this_jet_skew',
 'event_id_number_of_particles_in_this_jet_kurt',
 'event_id_number_of_particles_in_this_jet_q1',
 'event_id_number_of_particles_in_this_jet_q3',
 'event_id_jet_px_mean',
 'event_id_jet_px_max',
 'event_id_jet_px_min',
 'event_id_jet_px_sum',
 'event_id_jet_px_std',
 'event_id_jet_px_skew',
 'event_id_jet_px_kurt',
 'event_id_jet_px_q1',
 'event_id_jet_px_q3',
 'event_id_jet_py_mean',
 'event_id_jet_py_max',
 'event_id_jet_py_min',
 'event_id_jet_py_sum',
 'event_id_jet_py_std',
 'event_id_jet_py_skew',
 'event_id_jet_py_kurt',
 'event_id_jet_py_q1',
 'event_id_jet_py_q3',
 'event_id_jet_pz_mean',
 'event_id_jet_pz_max',
 'event_id_jet_pz_min',
 'event_id_jet_pz_sum',
 'event_id_jet_pz_std',
 'e

In [11]:
train_x = train[used_feat].reset_index(drop=True)
train_y = train['label'].reset_index(drop=True)
test_x = test[used_feat].reset_index(drop=True)

In [12]:
def multi_auc(y_true, y_score):
    score =  roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(np.argmax(y_score.reshape((-1,4)), axis=1)))
    return 'multi_auc', score, True

def multi_auc_oof(y_true, y_score):
    score =  roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(np.argmax(y_score, axis=1)))
    return score

In [13]:
preds = np.zeros((test_x.shape[0], 4))
oof_lgb = np.zeros((train_x.shape[0], 4))
scores = []

imp = pd.DataFrame()
imp['feat'] = used_feat

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=527)
for index, (tr_idx, va_idx) in enumerate(kfold.split(train_x, train_y)):
    print('*'*120)
    print('Fold %d' % (index+1))
    print('*'*120)
    X_train, y_train, X_valid, y_valid = train_x.iloc[tr_idx], train_y.iloc[tr_idx], train_x.iloc[va_idx], train_y.iloc[va_idx]
    eval_set = [(X_valid, y_valid)]
    model = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=2**6-1, max_depth=6, reg_alpha=0, reg_lambda=1.9, 
                                    max_bin=64, n_estimators=10000, objective='multiclass', num_class=4, 
                                    bagging_fraction=0.8, bagging_freq=5, min_child_samples=80, 
                                    feature_fraction=0.8, learning_rate=0.05, random_state=527, n_jobs=4)
#     model = ngb.NGBClassifier()
    
    model.fit(X_train, y_train, eval_set=eval_set,verbose=300, early_stopping_rounds=500,
#                   eval_metric=multi_auc, 
#                   categorical_feature=catlist
                 )
    
    imp['score%d' % (index+1)] = model.feature_importances_
    
    score = model.best_score_['valid_0']['multi_logloss']
    scores.append(score)
    print('fold %d round %d : score: %.6f | mean score %.6f' % (index+1, model.best_iteration_, score,np.mean(scores)))
    proba =  model.predict_proba(X_valid)
    print('auc: ',multi_auc_oof(y_valid, proba))
    print('\n\n')
    oof_lgb[va_idx] = proba
    preds += model.predict_proba(test_x)

************************************************************************************************************************
Fold 1
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.857927
[600]	valid_0's multi_logloss: 0.838436
[900]	valid_0's multi_logloss: 0.831069
[1200]	valid_0's multi_logloss: 0.826983
[1500]	valid_0's multi_logloss: 0.824636
[1800]	valid_0's multi_logloss: 0.822662
[2100]	valid_0's multi_logloss: 0.821613
[2400]	valid_0's multi_logloss: 0.821246
[2700]	valid_0's multi_logloss: 0.821017
[3000]	valid_0's multi_logloss: 0.821003
Early stopping, best iteration is:
[2780]	valid_0's multi_logloss: 0.820836


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 1 round 2780 : score: 0.820836 | mean score 0.820836
auc:  0.7702283255631476



************************************************************************************************************************
Fold 2
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.856531
[600]	valid_0's multi_logloss: 0.836005
[900]	valid_0's multi_logloss: 0.828952
[1200]	valid_0's multi_logloss: 0.824837
[1500]	valid_0's multi_logloss: 0.821829
[1800]	valid_0's multi_logloss: 0.820385
[2100]	valid_0's multi_logloss: 0.819494
[2400]	valid_0's multi_logloss: 0.818936
[2700]	valid_0's multi_logloss: 0.818718
[3000]	valid_0's multi_logloss: 0.818787
Early stopping, best iteration is:
[2554]	valid_0's multi_logloss: 0.818492


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 2 round 2554 : score: 0.818492 | mean score 0.819664
auc:  0.7716167566363751



************************************************************************************************************************
Fold 3
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.851868
[600]	valid_0's multi_logloss: 0.831313
[900]	valid_0's multi_logloss: 0.821974
[1200]	valid_0's multi_logloss: 0.817927
[1500]	valid_0's multi_logloss: 0.815605
[1800]	valid_0's multi_logloss: 0.814158
[2100]	valid_0's multi_logloss: 0.81295
[2400]	valid_0's multi_logloss: 0.812328
[2700]	valid_0's multi_logloss: 0.811792
[3000]	valid_0's multi_logloss: 0.811427
[3300]	valid_0's multi_logloss: 0.811457
Early stopping, best iteration is:
[3073]	valid_0's multi_logloss: 0.811327


...

fold 3 round 3073 : score: 0.811327 | mean score 0.816885
auc:  0.7725182085501521



************************************************************************************************************************
Fold 4
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.851459
[600]	valid_0's multi_logloss: 0.83051
[900]	valid_0's multi_logloss: 0.822347
[1200]	valid_0's multi_logloss: 0.818068
[1500]	valid_0's multi_logloss: 0.816102
[1800]	valid_0's multi_logloss: 0.813992
[2100]	valid_0's multi_logloss: 0.812938
[2400]	valid_0's multi_logloss: 0.81234
[2700]	valid_0's multi_logloss: 0.81183
[3000]	valid_0's multi_logloss: 0.811776
[3300]	valid_0's multi_logloss: 0.811685
Early stopping, best iteration is:
[2880]	valid_0's multi_logloss: 0.8116


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 4 round 2880 : score: 0.811600 | mean score 0.815563
auc:  0.7723779434755146



************************************************************************************************************************
Fold 5
************************************************************************************************************************
Training until validation scores don't improve for 500 rounds
[300]	valid_0's multi_logloss: 0.854332
[600]	valid_0's multi_logloss: 0.833935
[900]	valid_0's multi_logloss: 0.826427
[1200]	valid_0's multi_logloss: 0.822686
[1500]	valid_0's multi_logloss: 0.819941
[1800]	valid_0's multi_logloss: 0.817616
[2100]	valid_0's multi_logloss: 0.816372
[2400]	valid_0's multi_logloss: 0.815648
[2700]	valid_0's multi_logloss: 0.815146
[3000]	valid_0's multi_logloss: 0.814897
[3300]	valid_0's multi_logloss: 0.815158
Early stopping, best iteration is:
[3036]	valid_0's multi_logloss: 0.814875


LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
               class_weight=None, colsample_bytree=1.0, feature_fraction=0.8,
               importance_type='split', learning_rate=0.05, max_bin=64,
               max_depth=6, min_child_samples=80, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=10000, n_jobs=4, num_class=4,
               num_leaves=63, objective='multiclass', random_state=527,
               reg_alpha=0, reg_lambda=1.9, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

fold 5 round 3036 : score: 0.814875 | mean score 0.815426
auc:  0.7729694434467038





In [14]:
# auc:  0.7713910442312945 fold1
print(scores)
print('logloss:',np.mean(scores))
print('auc:', multi_auc_oof(train_y, oof_lgb))
print('acc:', accuracy_score(train_y.map(label_map), np.argmax(oof_lgb, axis=1)))
# logloss: 0.8394470890181213 auc: 0.7639691112376329 acc: 0.6468655840291672  featv0 0.75723 $
# logloss: 0.8386205747554791 auc: 0.7641546488063027 acc: 0.6471483155930496 featv1 0.75757 $
# logloss: 0.8385694282652787 auc: 0.7644872386334696 acc: 0.6476437690002342 527 527
# logloss: 0.83799263525929 auc: 0.7643224628185179 acc: 0.6474014276597635 featv2 527 527 0.75785 $
# logloss: 0.8230686055330182 auc: 0.7691376036021507 acc: 0.6545720386561364 featv3 0.76301 $
# logloss: 0.8153214789014147 auc: 0.772159018172669 acc: 0.6590661241810882 data_ 0.76654 $
# logloss: 0.8151705994830186 auc: 0.7722130382430792 acc: 0.6591495973094725 data_:1000 0.76649
# logloss: 0.8154789314371726 auc: 0.7720832773246235 acc: 0.6589530315555352 data_:1200 0.76588
# logloss: 0.8152194259449506 auc: 0.7722792166028103 acc: 0.6592411484825393 data_:800  0.76618
# logloss: 0.8157724809172372 auc: 0.7721843858406869 acc: 0.6591038217229392 data_:1400  0.76606
# logloss: 0.815500754326537 auc: 0.7718635245268304 acc: 0.6586245244051193 data_:1500  0.76559
# logloss: 0.8154257388311787 auc: 0.7719421140288143 acc: 0.6587456950753547 data_:1540

[0.8208357086305125, 0.8184917911679546, 0.8113269153190781, 0.811599542267215, 0.8148747367711344]
logloss: 0.8154257388311787
auc: 0.7719421140288143
acc: 0.6587456950753547


In [15]:
cm = confusion_matrix(train_y.map(label_map), np.argmax(oof_lgb, axis=1))
acc0 = cm[0][0] / cm[0].sum()
acc1 = cm[1][1] / cm[1].sum()
acc2 = cm[2][2] / cm[2].sum()
acc3 = cm[3][3] / cm[3].sum()
cm
acc0, acc1, acc2, acc3

array([[63164, 13872,  4753, 10958],
       [23202, 44246, 14640, 10482],
       [ 7328, 14178, 59345, 10273],
       [ 7566,  3767,  5715, 77888]], dtype=int64)

(0.6810355051915425,
 0.47797342551582583,
 0.6512554321583776,
 0.8204263925170642)

In [16]:
imp['score'] = imp['score1'] + imp['score2'] + imp['score3'] + imp['score4'] + imp['score5']
imp.sort_values(by='score', ascending=False)

,feat,score1,score2,score3,score4,score5,score
12,event_id_jet_px_sum,2204,2256,2542,2301,2333,11636
9,event_id_jet_px_mean,1795,1772,2066,2056,2081,9770
296,event_id_jet_id_particle_py_std_min,1727,1661,1855,1686,1753,8682
1210,event_id_jet_id_particle_eta_sinh_abs_q1_min,1723,1671,1847,1683,1713,8637
84,event_id_particle_category_-211_std,1671,1591,1779,1669,1820,8530
145,event_id_particle_category_211_std,1594,1575,1683,1703,1815,8370
377,event_id_jet_id_particle_pz_std_min,1581,1555,1792,1689,1630,8247
1137,event_id_jet_id_particle_phi_cos_abs_q3_max,1473,1414,1653,1542,1591,7673
1506,event_id_jet_energy_differ_min,1404,1306,1569,1422,1402,7103
1497,event_id_jet_eta_sinh_abs_min,1292,1293,1436,1381,1438,6840


In [17]:
# imp_array = np.array(imp.sort_values(by='score', ascending=False)['feat'])
# imp_array
# np.save('E:/jet_complex_data/imp.npy', imp_array)

In [18]:
test_jet = pd.read_csv('E:/jet_complex_data/complex_test_R04_jet.csv')

In [19]:
test['label'] = preds.argmax(axis=1)
test['label'] = test['label'].map(label_map_reverse).astype(np.int64)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
sub = test_jet.merge(test[['event_id', 'label']], on='event_id', how='left')[['jet_id', 'label']]
sub.rename(columns={'jet_id':'id'}, inplace = True)
sub

,id,label
0,cb3b5c6ea5e441e9b425fc1e6a4a00e6,5
1,838bf5c3f9bc4a2194f51b3c57f403cd,21
2,fbc245d8b90b4a5d804880e097244a67,1
3,f4c5dffa513a443fbbd27ca4c0fbe44f,21
4,74b105d4cb654c4d94da3fb88c8cd6a7,21
...,...,...
537944,e6d8625e7a804588bb5a03e97be8092e,21
537945,56f663b1a299408d905298bc1aef9ab3,4
537946,568150932e9f48cd86ccdb28faa29cff,4
537947,f3dc23154e874319a9d3775f8965975d,21


In [21]:
sub['label'].value_counts()
# 21    210942
# 1     157520
# 5     109405
# 4      60082

21    194142
1     135193
5     112430
4      96184
Name: label, dtype: int64

In [22]:
if not os.path.exists('E:/jet_complex_data/sub'):
    os.mkdir('E:/jet_complex_data/sub')
sub.to_csv('E:/jet_complex_data/sub/sub12-31_data_.1530.csv', index=False)

In [23]:
imp_top = imp.sort_values(by='score', ascending=False)['feat'][:5].values
imp_top = list(imp_top)
imp_top.append('label')
imp_top

['event_id_jet_px_sum',
 'event_id_jet_px_mean',
 'event_id_jet_id_particle_py_std_min',
 'event_id_jet_id_particle_eta_sinh_abs_q1_min',
 'event_id_particle_category_-211_std',
 'label']